In [228]:
import os
import glob
import pandas as pd

#create a dictionary to hold data frames
df_dict = {}

output_file = 'combined/lineup_data.csv'

for season_type in ['Regular Season', 'Playoffs']:
    
    #get csv files to read
    folder_path = season_type
    csv_files = glob.glob(os.path.join(folder_path,'*.csv'))
    
    # Initialize an empty list to hold df's
    df_list = []
    
    #loop over csv files
    for csv_file in csv_files:
        
        df = pd.read_csv(csv_file)
        
        #create columns for season
        season_name = csv_file.split("_")[2]
        season_strt_yr = int(season_name.split("-")[0])
        season_end_yr = int(season_strt_yr +1)
        df['SeasonName'] = season_name
        df['SeasonStrtYr'] = season_strt_yr
        df['SeasonEndYr'] = season_end_yr
        df['SeasonType'] = season_type
        
        #append dfs in a list
        df_list.append(df)
    
    #group together with pd.concat for better performance
    df_dict[season_type] = pd.concat(df_list, ignore_index=True)

all_data = pd.concat([df_dict['Regular Season'],df_dict['Playoffs']], ignore_index=True)

# Update the column order
cols = all_data.columns
season_col = ['SeasonName','SeasonStrtYr','SeasonEndYr','SeasonType']
new_order = list(cols[0:6]) + season_col
new_order = new_order + [col for col in cols if col not in new_order]
all_data = all_data[new_order]

#Split Name into 5 fields
all_data[['PlayerOne','PlayerTwo','PlayerThree','PlayerFour','PlayerFive']] = all_data['Name'].str.split(', ', expand=True)

all_data.to_csv(output_file, index = False)

In [157]:
lineup_data = all_data

# Read in player height csv
player_info = pd.read_csv("all_seasons.csv")
player_height = player_info[['player_name','season','player_height']]

# Join data with player height
for player in ['PlayerOne','PlayerTwo','PlayerThree','PlayerFour','PlayerFive']:
    lineup_data = pd.merge(lineup_data, player_height, left_on = [player,'SeasonName'],
                           right_on = ['player_name','season'], how = 'left', suffixes = ('',player))

# Rename Columns and remove unnecessary ones
lineup_data = lineup_data.rename(columns = {'player_height':'PlayerOneHeight',
                                           'player_heightPlayerTwo':'PlayerTwoHeight',
                                           'player_heightPlayerThree':'PlayerThreeHeight',
                                           'player_heightPlayerFour':'PlayerFourHeight',
                                           'player_heightPlayerFive':'PlayerFiveHeight'})

columns = ['EntityId','Name','TeamAbbreviation','SeasonName','SeasonEndYr','SeasonType','SecondsPlayed',
          'PlayerOne','PlayerOneHeight','PlayerTwo','PlayerTwoHeight','PlayerThree','PlayerThreeHeight',
           'PlayerFour','PlayerFourHeight','PlayerFive','PlayerFiveHeight']
lineup_data = lineup_data[columns]

In [174]:
lineup_data=all_data[['EntityId','Name','TeamAbbreviation','SeasonName','SeasonEndYr','SeasonType','SecondsPlayed',
          'PlayerOne','PlayerTwo','PlayerThree',
           'PlayerFour','PlayerFive']]
test = pd.melt(lineup_data,id_vars = ['EntityId','Name','TeamAbbreviation','SeasonName','SeasonEndYr','SeasonType','SecondsPlayed'],
                     var_name="Player", value_name = 'PlayerName')
test = pd.merge(test, player_height, left_on = ['PlayerName','SeasonName'],
                           right_on = ['player_name','season'], how = 'left')
test = test.sort_values(by = ['EntityId','player_height'], ascending = [True,False])
test

,EntityId,Name,TeamAbbreviation,SeasonName,SeasonEndYr,SeasonType,SecondsPlayed,Player,PlayerName,player_name,season,player_height
1004088,1000-1005-1508-1509-1883,"Shandon Anderson, Walt Williams, Maurice Taylo...",HOU,2000-01,2001,Regular Season,307.0,PlayerFour,Kelvin Cato,Kelvin Cato,2000-01,210.82
702553,1000-1005-1508-1509-1883,"Shandon Anderson, Walt Williams, Maurice Taylo...",HOU,2000-01,2001,Regular Season,307.0,PlayerThree,Maurice Taylor,Maurice Taylor,2000-01,205.74
401017,1000-1005-1508-1509-1883,"Shandon Anderson, Walt Williams, Maurice Taylo...",HOU,2000-01,2001,Regular Season,307.0,PlayerTwo,Walt Williams,Walt Williams,2000-01,203.20
99405,1000-1005-1508-1509-1883,"Shandon Anderson, Walt Williams, Maurice Taylo...",HOU,2000-01,2001,Regular Season,307.0,PlayerOne,Shandon Anderson,Shandon Anderson,2000-01,198.12
1305646,1000-1005-1508-1509-1883,"Shandon Anderson, Walt Williams, Maurice Taylo...",HOU,2000-01,2001,Regular Season,307.0,PlayerFive,Steve Francis,Steve Francis,2000-01,190.50
...,...,...,...,...,...,...,...,...,...,...,...,...
856136,915-952-958-960-962,"Rodney Rogers, Antoine Walker, Vitaly Potapenk...",BOS,2001-02,2002,Regular Season,69.0,PlayerThree,Vitaly Potapenko,Vitaly Potapenko,2001-02,208.28
1459229,915-952-958-960-962,"Rodney Rogers, Antoine Walker, Vitaly Potapenk...",BOS,2001-02,2002,Regular Season,69.0,PlayerFive,Walter McCarty,Walter McCarty,2001-02,208.28
554580,915-952-958-960-962,"Rodney Rogers, Antoine Walker, Vitaly Potapenk...",BOS,2001-02,2002,Regular Season,69.0,PlayerTwo,Antoine Walker,Antoine Walker,2001-02,205.74
252965,915-952-958-960-962,"Rodney Rogers, Antoine Walker, Vitaly Potapenk...",BOS,2001-02,2002,Regular Season,69.0,PlayerOne,Rodney Rogers,Rodney Rogers,2001-02,200.66


In [180]:
badNames = test.loc[test['player_height'].isna()]
badNames = badNames[['SeasonName','PlayerName','player_height']]
badNames = badNames.drop_duplicates()
badNames

,SeasonName,PlayerName,player_height
1053989,2017-18,PJ Tucker,NaN
1028504,2018-19,PJ Tucker,NaN
1498658,2015-16,Dorell Wright,NaN
590284,2012-13,Dwayne Jones,NaN
900551,2017-18,Ty Lawson,NaN
591059,2012-13,Tracy McGrady,NaN
1032646,2018-19,Kevin Knox,NaN
967027,2016-17,Juan Hernangomez,NaN
1414096,2021-22,Xavier Tillman,NaN
1497949,2015-16,John Holland,NaN


In [227]:
condition1 = player_height['player_name'].str.contains('Tucker')
condition2 = player_height['season'] == '2017-18'
player_height[condition1 & condition2]



,player_name,season,player_height
9790,P.J. Tucker,2017-18,198.12


In [162]:
# Updates to be made:
# PJ Tucker = P.J. Tucker
# Kevin Knox = Kevin Knox II
# Juan Hernangomez = Juancho
# Xavier Tillman = Xavier Tillman Sr.
# Patrick Baldwin = Patrick Baldwin Jr


# Players only playing playoffs
# Dorell Wright played in playoffs. Should copy 2014-2015 version
# Dwayne Jones only played in playoffs. Copy 2009-2010
# Ty Lawson only playoffs. Copy 2016-17
# Tmac
#John Holland, Tristan Thompson


Original DataFrame:
   ID     Name  Math_Score  English_Score
0   1    Alice          90             88
1   2      Bob          85             92
2   3  Charlie          95             89

Reshaped DataFrame:
   ID     Name        Subject  Score
0   1    Alice     Math_Score     90
1   2      Bob     Math_Score     85
2   3  Charlie     Math_Score     95
3   1    Alice  English_Score     88
4   2      Bob  English_Score     92
5   3  Charlie  English_Score     89


In [222]:
condition1 = test['PlayerName'].str.contains('Patrick Baldwin Jr.')
condition2 = test['SeasonName'] == '2022-23'
condition3 = test['SeasonType'] == 'Playoffs'
test[condition1 & condition2]



#815296	2021-22	MJ Walker	NaN
#713750	2019-20	TJ Leaf	NaN
#1326128	2018-19	Marcus Morris	NaN
#1502512	2017-18	Marcus Morris	NaN
#110443	2019-20	Michael Frazier	NaN
#1312417	2019-20	Charles Brown Jr.	NaN
#1183422	2019-20	Jaylen Adams	NaN
#422644	2018-19	Walter Lemon Jr.	NaN
#152587	2017-18	Walter Lemon Jr.	NaN
#592914	2022-23	DaQuan Jeffries	NaN
#1115075	2021-22	Aleksej Pokusevski	NaN
#1186596	2021-22	Luca Vildoza	NaN
#773143	2002-03	Nene	NaN

,EntityId,Name,TeamAbbreviation,SeasonName,SeasonEndYr,SeasonType,SecondsPlayed,Player,PlayerName,player_name,season,player_height
1473124,1626172-1628978-1629660-1630237-1631116,"Kevon Looney, Donte DiVincenzo, Ty Jerome, Ant...",GSW,2022-23,2023,Regular Season,104.0,PlayerFive,Patrick Baldwin Jr.,NaN,NaN,NaN
1171363,1626172-1628978-1629660-1631116-202691,"Kevon Looney, Donte DiVincenzo, Ty Jerome, Pat...",GSW,2022-23,2023,Regular Season,420.0,PlayerFour,Patrick Baldwin Jr.,NaN,NaN,NaN
1472930,1626172-1628978-1629673-1630228-1631116,"Kevon Looney, Donte DiVincenzo, Jordan Poole, ...",GSW,2022-23,2023,Regular Season,332.0,PlayerFive,Patrick Baldwin Jr.,NaN,NaN,NaN
1472997,1626172-1628978-1629673-1630237-1631116,"Kevon Looney, Donte DiVincenzo, Jordan Poole, ...",GSW,2022-23,2023,Regular Season,223.0,PlayerFive,Patrick Baldwin Jr.,NaN,NaN,NaN
1473156,1626172-1628978-1629673-1630541-1631116,"Kevon Looney, Donte DiVincenzo, Jordan Poole, ...",GSW,2022-23,2023,Regular Season,81.0,PlayerFive,Patrick Baldwin Jr.,NaN,NaN,NaN
1171630,1626172-1628978-1629673-1631116-202691,"Kevon Looney, Donte DiVincenzo, Jordan Poole, ...",GSW,2022-23,2023,Regular Season,77.0,PlayerFour,Patrick Baldwin Jr.,NaN,NaN,NaN
1171403,1626172-1628978-1629673-1631116-203110,"Kevon Looney, Donte DiVincenzo, Jordan Poole, ...",GSW,2022-23,2023,Regular Season,314.0,PlayerFour,Patrick Baldwin Jr.,NaN,NaN,NaN
1473148,1626172-1629660-1629673-1630237-1631116,"Kevon Looney, Ty Jerome, Jordan Poole, Anthony...",GSW,2022-23,2023,Regular Season,88.0,PlayerFive,Patrick Baldwin Jr.,NaN,NaN,NaN
1473099,1626172-1629660-1629673-1630541-1631116,"Kevon Looney, Ty Jerome, Jordan Poole, Moses M...",GSW,2022-23,2023,Regular Season,129.0,PlayerFive,Patrick Baldwin Jr.,NaN,NaN,NaN
1472893,1626172-1629660-1630228-1630541-1631116,"Kevon Looney, Ty Jerome, Jonathan Kuminga, Mos...",GSW,2022-23,2023,Regular Season,426.0,PlayerFive,Patrick Baldwin Jr.,NaN,NaN,NaN
